In [1]:
from magent2.environments import battle_v4, adversarial_pursuit_v4, tiger_deer_v4
from pettingzoo.utils import random_demo
import torch
import time
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from dqn_basic import DQN_Basic
from collections import namedtuple, deque
import numpy as np
# import matplotlib.pyplot as plt
from AnimalLSTM import Animal
from pathlib import Path
from collections import deque
import datetime, os
from metric_logger import MetricLogger
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# frame_size = (128, 128)
max_cycles = 300
# total_episodes = 50
map_size = 50

# env = tiger_deer_v4.env(map_size=map_size, minimap_mode=False, render_mode='human', tiger_step_recover=-0.1, deer_attacked=-0.1, max_cycles=max_cycles, extra_features=False)
env = tiger_deer_v4.env(map_size=map_size, minimap_mode=False, render_mode='rgb_array', max_cycles=max_cycles, extra_features=False)
env.reset(seed=None)

print(env.action_spaces['deer_0'].n)
number_of_deer = len([x for x in env.agents if 'deer' in x])
print("Number of Deer:",number_of_deer)
number_of_tigers = len([x for x in env.agents if 'tiger' in x])
print("Number of Tigers:", number_of_tigers)
deer_observation_shape = env.observation_spaces['deer_0'].shape
deer_observation_shape = tuple(np.roll(deer_observation_shape,1))
# deer_action_space = env.action_spaces['deer_0'].n
deer_action_space = 3

tiger_observation_shape = env.observation_spaces['tiger_0'].shape
tiger_observation_shape = tuple(np.roll(tiger_observation_shape,1))
# tiger_action_space = env.action_spaces['tiger_0'].n
tiger_action_space = 6

print("Deer Observations:", deer_observation_shape, "Tiger Observations:",tiger_observation_shape)
assert deer_observation_shape != None
assert tiger_observation_shape != None
# 'rgb_array' or 'human'
# random_demo(env, render=False , episodes=1)
# 'rgb_array'
# pygame.quit()

5
Number of Deer: 80
Number of Tigers: 16
Deer Observations: (5, 13, 13) Tiger Observations: (5, 13, 13)


c:\Users\Scott\a3\lib\site-packages\pettingzoo\utils\wrappers\base.py:64: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn(
c:\Users\Scott\a3\lib\site-packages\pettingzoo\utils\wrappers\base.py:50: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn(


In [3]:
deer_action_mapping = {
    0: {0: 0, 1: 1, 3: 3, 4: 4},
    1: {0: 1, 1: 4, 3: 0, 4: 3},
    2: {0: 3, 1: 0, 3: 4, 4: 1},
}
def deer_action_mapper(new_unmapped_action, prev_actual_action, size_of_actions=5):
    global deer_action_mapping
    return deer_action_mapping.get(new_unmapped_action, {}).get(prev_actual_action, np.random.choice(np.delete(np.arange(0, size_of_actions), 2)))

tiger_action_mapping = {
    0: {0: 0, 1: 1, 3: 3, 4: 4},
    1: {0: 1, 1: 4, 3: 0, 4: 3},
    2: {0: 3, 1: 0, 3: 4, 4: 1},
    3: {0: 5, 1: 6, 3: 7, 4: 8},
    4: {0: 6, 1: 8, 3: 5, 4: 7},
    5: {0: 7, 1: 5, 3: 8, 4: 6},
}

tiger_direction_mapping = {
    0: 0,
    1: 1,
    3: 3,
    4: 4,
    5: 0,
    6: 1,
    7: 3,
    8: 4,
}

def tiger_action_mapper(new_action, prev_action, size_of_actions=5): # slower
    global tiger_action_mapping, tiger_direction_mapping
    direction = tiger_direction_mapping.get(prev_action, 0)
    return tiger_action_mapping.get(new_action, {}).get(direction, 0)

def mask_observations(observations, prev_actual_action):
    current_position = (observations.shape[1] // 2, observations.shape[2] // 2)
    direction = tiger_direction_mapping.get(prev_actual_action, 0)
    if direction == 0:  # Up
        observations[:, current_position[0]:, :] = 0
    elif direction == 1:  # Left
        observations[:, :, current_position[1]:] = 0
    elif direction == 3:  # Right
        observations[:, :, :current_position[1]] = 0
    elif direction == 4:  # Down
        observations[:, :current_position[0]+1, :] = 0
    else:
        raise ValueError("Direction mapping issue for mask observations")
    # print("direction: ", direction, " observations: ", observations)
    

In [4]:
def get_agent_type_from_agent_name(agent):
    if 'tiger' in agent:
        return 'tiger'
    return 'deer'

In [6]:
def generate_first_action(agentType):
    if 'tiger' in agentType:
        # action = np.random.randint(0,tiger_action_space,(1,))[0]
        action = np.random.choice(np.arange(0, 6))
        previous_actual_action = np.random.choice(np.delete(np.arange(0, 9), 2))
        actual_action = tiger_action_mapper(action, previous_actual_action)
    else:
        # action = np.random.randint(0,deer_action_space,(1,))[0]
        action = np.random.choice(np.arange(0, 3))
        previous_actual_action = np.random.choice(np.delete(np.arange(0, 5), 2))
        actual_action = deer_action_mapper(action, previous_actual_action)
    
    return action, actual_action, previous_actual_action

In [7]:
def change_active_model(activeDeer, activeTiger, deerCurrentSteps, tigerCurrentSteps, steps_to_switch_at):
    if tigerCurrentSteps < (deerCurrentSteps//steps_to_switch_at)*steps_to_switch_at:
        if not activeTiger:
            print("Switching to Tiger")
        return False, True
    if deerCurrentSteps < (tigerCurrentSteps//steps_to_switch_at)*steps_to_switch_at:
        if not activeDeer:
            print("Switching to Deer")
        return True, False
    return activeDeer, activeTiger

## Train Model

### Make Models and Loggers

In [8]:
use_cuda = torch.cuda.is_available()
print(f"Using CUDA: {use_cuda}")
print()

deer_save_dir = Path("deer_checkpoints") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
deer_save_dir.mkdir(parents=True)

tiger_save_dir = Path("tiger_checkpoints") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
tiger_save_dir.mkdir(parents=True)

modelType = 'basic'

deer = Animal(state_dim=deer_observation_shape, action_dim=deer_action_space, save_dir=deer_save_dir)
tiger = Animal(state_dim=tiger_observation_shape, action_dim=tiger_action_space, save_dir=tiger_save_dir)

deer_logger = MetricLogger(deer_save_dir, animal='deer')
tiger_logger = MetricLogger(tiger_save_dir, animal='tiger')


# active training flags
activeDeer = True
activeTiger = False


# Switch the training set
switching = True
steps_to_switch_at = 15000
total_steps = 90000

# divide steps to switch at to actually switch at the proper requested switching steps
# steps_to_switch_at = steps_to_switch_at//2
# if switching enabled, make sure no equivalence between active variables
if switching:
    if activeDeer == activeTiger:
        activeTiger = not activeTiger

Using CUDA: True



c:\Users\Scott\a3\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


### Train Model

In [9]:
print("Number of Tigers:", number_of_tigers)
print("Number of Deer:",number_of_deer)

e = -1
while deer.curr_step < total_steps or tiger.curr_step < total_steps:
    e = e + 1
# for e in range(total_episodes):
    # print("Episode: ", e)
    env.reset(seed=None)
    agents_prev_data = {}
    # Play the game!
    for agent in env.agent_iter():
        
        observation, reward, termination, truncation, info = env.last()
        observation = np.transpose(observation, (2,0,1))
        done = termination or truncation
        agentType = get_agent_type_from_agent_name(agent)

        # instantiate previous data
        if agent not in agents_prev_data.keys():
            action, actual_action, prev_actual_action = generate_first_action(agentType)
            mask_observations(observation, prev_actual_action)
            agents_prev_data[agent] = [observation, action, done, actual_action, None, None, reward,  info]
            env.step(actual_action)
            continue
        
        prev_observation, prev_action, prev_done, prev_actual_action, hidden, cell = agents_prev_data[agent][0:6]
        mask_observations(observation, prev_actual_action)
        
        if not prev_done:
            if 'tiger' in agentType:
                if activeTiger:
                    tiger.cache(prev_observation, observation, hidden, cell, prev_action, reward, done)
                    #learn
                    q, loss = tiger.learn()
                    # logging
                    tiger_logger.log_step(reward, loss, q)
                action, (hidden, cell) = tiger.act(observation, hidden, cell, activeTiger)
                actual_action = tiger_action_mapper(action, prev_actual_action)
            else:
                if activeDeer:
                    deer.cache(prev_observation, observation, hidden, cell, prev_action, reward, done)
                    #learn
                    q, loss = deer.learn()
                    # logging
                    deer_logger.log_step(reward, loss, q)
                action, (hidden, cell) = deer.act(observation, hidden, cell, activeDeer)
                actual_action = deer_action_mapper(action, prev_actual_action)
        
        if done:
            action = None
            actual_action = None
        
        agents_prev_data[agent] = [observation, action, done, actual_action, hidden, cell, reward,  info]
        env.step(actual_action)
        

    deer_logger.log_episode()
    tiger_logger.log_episode()

    # if e % 5 == 0:
    if activeDeer:
        # print("Deer - Min Possible Reward ", -number_of_deer)
        deer_logger.record(episode=e, epsilon=deer.exploration_rate, step=deer.curr_step)
    if activeTiger:
        # print("Tiger - Max Possible Reward ", number_of_deer)
        tiger_logger.record(episode=e, epsilon=tiger.exploration_rate, step=tiger.curr_step)
    if switching:
        activeDeer, activeTiger = change_active_model(activeDeer, activeTiger, deer.curr_step, tiger.curr_step, steps_to_switch_at)

# log last episode
# if total_episodes != 0:
deer_logger.record(episode=e, epsilon=deer.exploration_rate, step=deer.curr_step)
tiger_logger.record(episode=e, epsilon=tiger.exploration_rate, step=tiger.curr_step)

Number of Tigers: 16
Number of Deer: 80
Animal saved to deer_checkpoints\2023-04-25T11-21-14\animal_net_0.chkpt at step 0


c:\Users\Scott\OneDrive - Georgia Institute of Technology\Georgia Tech\GTSR\Underwater Platforms\Python_Code\TheLearningShoal\The-Learning-Shoal\src\tests\scott_tests\AnimalLSTM.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hidden = torch.tensor(hidden, device=self.device) #.unsqueeze(0)
c:\Users\Scott\OneDrive - Georgia Institute of Technology\Georgia Tech\GTSR\Underwater Platforms\Python_Code\TheLearningShoal\The-Learning-Shoal\src\tests\scott_tests\AnimalLSTM.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cell = torch.tensor(cell, device=self.device) #.unsqueeze(0)


Episode 0 - deer - Step 18493 - Epsilon 0.995387420193517 - Mean Reward -27.0 - Mean Length 18493.0 - Mean Loss 0.001 - Mean Q Value 0.024 - Time Delta 47.651 - Time 2023-04-25T11:22:03
Switching to Tiger
Animal saved to tiger_checkpoints\2023-04-25T11-21-14\animal_net_0.chkpt at step 0
Episode 1 - tiger - Step 1865 - Epsilon 0.9995338586193204 - Mean Reward 14.0 - Mean Length 932.5 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 68.646 - Time 2023-04-25T11:22:24
Episode 2 - tiger - Step 4518 - Epsilon 0.9988711375038438 - Mean Reward 23.333 - Mean Length 1506.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 29.799 - Time 2023-04-25T11:22:54
Episode 3 - tiger - Step 7254 - Epsilon 0.9981881431710098 - Mean Reward 29.75 - Mean Length 1813.5 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 27.643 - Time 2023-04-25T11:23:21
Episode 4 - tiger - Step 9268 - Epsilon 0.9976856818832875 - Mean Reward 32.0 - Mean Length 1853.6 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 23.898 - Time 2023-0

<Figure size 640x480 with 0 Axes>

### Visualize Trained Model

In [12]:
env = tiger_deer_v4.env(map_size=map_size, minimap_mode=False, render_mode='human', max_cycles=max_cycles, extra_features=False)
example_episodes = 10
for e in range(example_episodes):
    print("Episode: ", e)
    env.reset(seed=None)
    # Play the game!
    agents_prev_data = {}
    init_agent = None
    for agent in env.agent_iter():
        if init_agent is None:
            init_agent = agent
        else:
            if init_agent == agent:
                time.sleep(0.01)

        observation, reward, termination, truncation, info = env.last()
        observation = np.transpose(observation, (2,0,1))
        done = termination or truncation
        agentType = get_agent_type_from_agent_name(agent)

        # instantiate previous data
        if agent not in agents_prev_data.keys():
            action, actual_action, prev_actual_action = generate_first_action(agentType)
            mask_observations(observation, prev_actual_action)
            agents_prev_data[agent] = [observation, action, done, actual_action, None, None, reward,  info]
            env.step(actual_action)
            continue

        prev_observation, prev_action, prev_done, prev_actual_action, hidden, cell = agents_prev_data[agent][0:6]
        mask_observations(observation, prev_actual_action)

        if not done:
            if 'tiger' in agentType:
                # tiger.cache(prev_observation, observation, prev_action, reward, done, agentName=agent)
                action, (hidden, cell) = tiger.act(observation, hidden, cell, activeTiger)
                actual_action = tiger_action_mapper(action, prev_actual_action)
            else:
                # deer.cache(prev_observation, observation, prev_action, reward, done, agentName=agent)
                # action =
                action, (hidden, cell) = deer.act(observation, hidden, cell, activeDeer)
                actual_action = deer_action_mapper(action, prev_actual_action)
        else:
            action = None
            actual_action = None
        
        agents_prev_data[agent] = [observation, action, done, actual_action, hidden, cell, reward,  info]
        env.step(actual_action)
    

Episode:  0


c:\Users\Scott\OneDrive - Georgia Institute of Technology\Georgia Tech\GTSR\Underwater Platforms\Python_Code\TheLearningShoal\The-Learning-Shoal\src\tests\scott_tests\AnimalLSTM.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hidden = torch.tensor(hidden, device=self.device) #.unsqueeze(0)
c:\Users\Scott\OneDrive - Georgia Institute of Technology\Georgia Tech\GTSR\Underwater Platforms\Python_Code\TheLearningShoal\The-Learning-Shoal\src\tests\scott_tests\AnimalLSTM.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cell = torch.tensor(cell, device=self.device) #.unsqueeze(0)


Episode:  1
Episode:  2
Episode:  3
Episode:  4
Episode:  5
Episode:  6


SystemExit: 

c:\Users\Scott\a3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
